In [3]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import os
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver

## Scrapping UIN Sunan Kalijaga Yogyakarta

In [23]:
# URL dari halaman yang ingin di-scrape
base_url = "https://digilib.uin-suka.ac.id/view/divisions/jur=5Fmat/2024.html"  # Ganti dengan URL target

# Fungsi untuk melakukan scraping ID dari link
def scrape_ids(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Cari semua elemen <a> dengan href mengarah ke https://
        links = soup.find_all('a', href=True)
        
        # Filter hanya link yang sesuai pola dan ekstrak ID
        ids = []
        for link in links:
            href = link['href']
            if href.startswith("https://digilib.uin-suka.ac.id/"):
                # Ambil ID dari URL (angka setelah domain)
                id_value = href.split('/')[-2]  # Mengambil ID sebelum tanda "/"
                ids.append(id_value)
        
        return ids
    else:
        print(f"Failed to fetch page. Status code: {response.status_code}")
        return []

# Fungsi untuk mengambil data detail dari halaman berdasarkan ID
def scrape_detail(id_value):
    # Bentuk URL menggunakan pola yang benar
    url = f"https://digilib.uin-suka.ac.id/id/eprint/{id_value}/"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Tambahkan proses scraping di sini sesuai kebutuhan
        
        # Ambil paragraf abstrak
        paragraph = soup.find('p', style="text-align: justify; margin: 1em auto 0em auto")  # Perbaiki selector
        abstract = paragraph.text.strip() if paragraph else None
        if abstract:
            abstract = f"{abstract}"
        
        # Ambil tahun
        year_tag = soup.find('span', class_='person_name')
        if year_tag and year_tag.next_sibling:
            sibling_text = year_tag.next_sibling.strip()
            match = re.search(r'\((\d{4})\)', sibling_text)
            year = match.group(1) if match else None
        else:
            year = None

        # Ambil universitas
        paragraphs = soup.find_all('p')
        university = None
        for para in paragraphs:
            if "UIN" in para.text:
                university = para.text.split(",")[-1].strip()
                break
                    
        # Ambil judul
        title_tag = soup.find('em')
        title = title_tag.text.strip() if title_tag else None

        return {
            "id": id_link,
            "year": year,
            "university": university,
            "title": title,
            "abstract": abstract
        }
    else:
        print(f"Failed to fetch page for ID {id_link}. Status code: {response.status_code}")
        return {"id": id_link, "year": None, "university": None, "title": None, "abstract": None}

# Langkah 1: Scrape semua ID dari halaman utama
ids = scrape_ids(base_url)

# Langkah 2: Filter hanya ID berupa angka, mulai dari nomor 9 hingga maksimum 100
filtered_ids = [item for item in ids if item.isdigit()]
ids_from_0_to_100 = filtered_ids[0:100]  # Indeks dimulai dari 0

# Langkah 3: Ambil detail dari setiap link ID
result = []
for id_link in ids_from_0_to_100:
    details = scrape_detail(id_link)
    result.append(details)

# Langkah 4: Masukkan data ke dalam DataFrame
df_mtkuinsuka = pd.DataFrame(result)

# Langkah 5: Tampilkan DataFrame
print(df_mtkuinsuka.head())

      id  year                      university  \
0  67408  2024  UIN SUNAN KALIJAGA YOGYAKARTA.   
1  65638  2024  UIN SUNAN KALIJAGA YOGYAKARTA.   
2  64968  2024  UIN SUNAN KALIJAGA YOGYAKARTA.   
3  65640  2024  UIN SUNAN KALIJAGA YOGYAKARTA.   
4  65481  2024  UIN SUNAN KALIJAGA YOGYAKARTA.   

                                               title  \
0  OPTIMISASI ANALISIS REGRESI LASSO DENGAN ALGOR...   
1  PEMBENTUKAN PORTOFOLIO ROBUST MEAN-VARIANCE ME...   
2  PERBANDINGAN MODEL MIXED GEOGRAPHICALLY WEIGHT...   
3  PEMODELAN INDEKS GINI RATIO INDONESIA DENGAN P...   
4  PERAMALAN INDEKS HARGA SAHAM MENGGUNAKAN METOD...   

                                            abstract  
0  Linear regression is one of the modeling metho...  
1  Portfolio is a collection of several assets. T...  
2  The employment size in Indonesia, particularly...  
3  Dynamic panel data regression model is a model...  
4  Artificial Neural Network (ANN) adalah model l...  


In [24]:
# Nama file CSV
filename = './csv/mtk_uinsuka2.csv'

# Pastikan folder tempat menyimpan file ada
os.makedirs(os.path.dirname(filename), exist_ok=True)

try:
    # Cek apakah file sudah ada
    if os.path.exists(filename):
        # Jika file sudah ada, baca file CSV ke dalam DataFrame
        existing_data = pd.read_csv(filename, encoding="utf-8-sig")
        
        # Tambahkan data baru ke DataFrame yang sudah ada
        updated_data = pd.concat([existing_data, df_mtkuinsuka], ignore_index=True)
    else:
        # Jika file belum ada, data baru menjadi DataFrame awal
        updated_data = df_mtkuinsuka

    # Simpan DataFrame gabungan ke file CSV
    updated_data.to_csv(filename, index=False, encoding="utf-8-sig")

    print(f"DataFrame telah diperbarui dan disimpan ke file '{filename}'")

except Exception as e:
    print(f"Terjadi kesalahan: {e}")

DataFrame telah diperbarui dan disimpan ke file './csv/mtk_uinsuka2.csv'


## Scrapping UIN Sunan Gunung Djati

In [39]:
# URL dari halaman yang ingin di-scrape
base_url = "https://digilib.uinsgd.ac.id/view/divisions/prodi=5Fmatematika/2024.html"  # Ganti dengan URL target

# Fungsi untuk melakukan scraping ID dari link
def scrape_ids(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Cari semua elemen <a> dengan href mengarah ke https://
        links = soup.find_all('a', href=True)
        
        # Filter hanya link yang sesuai pola dan ekstrak ID
        ids = []
        for link in links:
            href = link['href']
            if href.startswith("https://digilib.uinsgd.ac.id/"):
                # Ambil ID dari URL (angka setelah domain)
                id_value = href.split('/')[-2]  # Mengambil ID sebelum tanda "/"
                ids.append(id_value)
        
        return ids
    else:
        print(f"Failed to fetch page. Status code: {response.status_code}")
        return []

# Fungsi untuk mengambil data detail dari halaman berdasarkan ID
def scrape_detail(id_value):
    # Bentuk URL menggunakan pola yang benar
    url = f"https://digilib.uinsgd.ac.id/{id_value}/"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Tambahkan proses scraping di sini sesuai kebutuhan
        
        # Ambil paragraf abstrak
        paragraph = soup.find('p', style="text-align: justify; margin: 1em auto 0em auto; line-height:1.5em; white-space:pre-line")  # Perbaiki selector
        abstract = paragraph.text.strip() if paragraph else None
        if abstract:
            abstract = f"{abstract}"
        
        # Ambil tahun
        year_tag = soup.find('span', class_='person_name')
        if year_tag and year_tag.next_sibling:
            sibling_text = year_tag.next_sibling.strip()
            match = re.search(r'\((\d{4})\)', sibling_text)
            year = match.group(1) if match else None
        else:
            year = None

        # Ambil universitas
        paragraphs = soup.find_all('p')
        university = None
        for para in paragraphs:
            if "UIN" in para.text:
                university = para.text.split(",")[-1].strip()
                break
                    
        # Ambil judul
        title_tag = soup.find('em')
        title = title_tag.text.strip() if title_tag else None

        return {
            "id": id_link,
            "year": year,
            "university": university,
            "title": title,
            "abstract": abstract
        }
    else:
        print(f"Failed to fetch page for ID {id_link}. Status code: {response.status_code}")
        return {"id": id_link, "year": None, "university": None, "title": None, "abstract": None}

# Langkah 1: Scrape semua ID dari halaman utama
ids = scrape_ids(base_url)

# Langkah 2: Filter hanya ID berupa angka, mulai dari nomor 9 hingga maksimum 100
filtered_ids = [item for item in ids if item.isdigit()]
ids_from_0_to_100 = filtered_ids[0:100]  # Indeks dimulai dari 0

# Langkah 3: Ambil detail dari setiap link ID
result = []
for id_link in ids_from_0_to_100:
    details = scrape_detail(id_link)
    result.append(details)

# Langkah 4: Masukkan data ke dalam DataFrame
df_mtkuinsgd = pd.DataFrame(result)

# Langkah 5: Tampilkan DataFrame
print(df_mtkuinsgd.head())

      id  year                         university  \
0  94061  2024    UIN Sunan Gunung Djati Bandung.   
1  95299  2024      UIN Sunan Gung Djati Bandung.   
2  93678  2024  : UIN Sunan Gunung Djati Bandung.   
3  90307  2024    UIN Sunan Gunung Djati Bandung.   
4  94198  2024    UIN Sunan Gunung Djati Bandung.   

                                               title  \
0  Penyelesaian masalah transportasi multi-objekt...   
1  Algoritma el-gamal pada teknik kriptografi aff...   
2  Identifikasi faktor-faktor yang memengaruhi pe...   
3  Bifurkasi Backward pada model HIV dalam tubuh ...   
4  Nilai total ketakteraturan sisi Modular pada G...   

                                            abstract  
0  Pembangunan berkelanjutan diperlakukan sebagai...  
1  Skripsi ini membahas kriptografi kunci publik ...  
2  Analisis survival adalah prosedur statistika u...  
3  Human Immunodeficiency Virus (HIV) telah menja...  
4                                               None  


In [40]:
# Nama file CSV
filename = './csv/mtk_uinsgd2.csv'

# Cek apakah file sudah ada
if os.path.exists(filename):
    # Jika file sudah ada, baca file CSV ke dalam DataFrame
    existing_data = pd.read_csv(filename)
    
    # Tambahkan data baru ke DataFrame yang sudah ada
    updated_data = pd.concat([existing_data, df_mtkuinsgd], ignore_index=True)
else:
    # Jika file belum ada, data baru menjadi DataFrame awal
    updated_data = df_mtkuinsgd

# Simpan DataFrame gabungan ke file CSV
updated_data.to_csv(filename, index=False)

print(f"DataFrame telah diperbarui dan disimpan ke file '{filename}'")

DataFrame telah diperbarui dan disimpan ke file './csv/mtk_uinsgd2.csv'


## Scrapping UIN Walisongo

In [ ]:
# URL dari halaman yang ingin di-scrape
base_url = "https://eprints.walisongo.ac.id/view/divisions/jur=5Fmat/2019.html"  # Ganti dengan URL target

# Fungsi untuk melakukan scraping ID dari link
def scrape_ids(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Cari semua elemen <a> dengan href mengarah ke https://
        links = soup.find_all('a', href=True)
        
        # Filter hanya link yang sesuai pola dan ekstrak ID
        ids = []
        for link in links:
            href = link['href']
            if href.startswith("https://eprints.walisongo.ac.id"):
                # Ambil ID dari URL (angka setelah domain)
                id_value = href.split('/')[-2]  # Mengambil ID sebelum tanda "/"
                ids.append(id_value)
        
        return ids
    else:
        print(f"Failed to fetch page. Status code: {response.status_code}")
        return []

# Fungsi untuk mengambil data detail dari halaman berdasarkan ID
def scrape_detail(id_value):
    # Bentuk URL menggunakan pola yang benar
    url = f"https://digilib.uinsgd.ac.id/id/eprint/{id_value}/"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Tambahkan proses scraping di sini sesuai kebutuhan
        
        # Ambil paragraf abstrak
        paragraph = soup.find('p.ep_field_para', style="text-align: left; margin: 1em auto 0em auto")  # Perbaiki selector
        abstract = paragraph.text.strip() if paragraph else None
        if abstract:
            abstract = f"{abstract}"
        
        # Ambil tahun
        year_tag = soup.find('p', class_='person_name')
        if year_tag and year_tag.next_sibling:
            sibling_text = year_tag.next_sibling.strip()
            match = re.search(r'\((\d{4})\)', sibling_text)
            year = match.group(1) if match else None
        else:
            year = None

        # Ambil universitas
        paragraphs = soup.find_all('p')
        university = None
        for para in paragraphs:
            if "UIN" in para.text:
                university = para.text.split(",")[-1].strip()
                break
                    
        # Ambil judul
        title_tag = soup.find('title')
        title = title_tag.text.strip() if title_tag else None

        return {
            "id": id_link,
            "year": year,
            "university": university,
            "title": title,
            "abstract": abstract
        }
    else:
        print(f"Failed to fetch page for ID {id_link}. Status code: {response.status_code}")
        return {"id": id_link, "year": None, "university": None, "title": None, "abstract": None}

# Langkah 1: Scrape semua ID dari halaman utama
ids = scrape_ids(base_url)

# Langkah 2: Filter hanya ID berupa angka, mulai dari nomor 9 hingga maksimum 100
filtered_ids = [item for item in ids if item.isdigit()]
ids_from_0_to_100 = filtered_ids[0:100]  # Indeks dimulai dari 0

# Langkah 3: Ambil detail dari setiap link ID
result = []
for id_link in ids_from_0_to_100:
    details = scrape_detail(id_link)
    result.append(details)

# Langkah 4: Masukkan data ke dalam DataFrame
df_mtkuinwls = pd.DataFrame(result)

# Langkah 5: Tampilkan DataFrame
print(df_mtkuinwls.head())

Failed to fetch page for ID 10388. Status code: 404
Failed to fetch page for ID 12293. Status code: 404
Failed to fetch page for ID 12292. Status code: 404
Failed to fetch page for ID 10387. Status code: 404
      id  year university title abstract
0  10388  None       None  None     None
1  12293  None       None  None     None
2  12292  None       None  None     None
3  10387  None       None  None     None


## Scrapping UIN Siber Syekh Nurjati Cirebon

In [7]:
# URL dari halaman yang ingin di-scrape
base_url = "https://repository.syekhnurjati.ac.id/view/subjects/QA.html"  # Ganti dengan URL target

# Fungsi untuk melakukan scraping ID dari link
def scrape_ids(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Cari semua elemen <a> dengan href mengarah ke https://
        links = soup.find_all('a', href=True)
        
        # Filter hanya link yang sesuai pola dan ekstrak ID
        ids = []
        for link in links:
            href = link['href']
            if href.startswith("https://repository.syekhnurjati.ac.id"):
                # Ambil ID dari URL (angka setelah domain)
                id_value = href.split('/')[-2]  # Mengambil ID sebelum tanda "/"
                ids.append(id_value)
        
        return ids
    else:
        print(f"Failed to fetch page. Status code: {response.status_code}")
        return []

# Fungsi untuk mengambil data detail dari halaman berdasarkan ID
def scrape_detail(id_value):
    # Bentuk URL menggunakan pola yang benar
    url = f"https://repository.syekhnurjati.ac.id/{id_value}/"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Tambahkan proses scraping di sini sesuai kebutuhan
        
        # Ambil paragraf abstrak
        paragraph = soup.find('p', style="text-align: left; margin: 1em auto 0em auto")  # Perbaiki selector
        abstract = paragraph.text.strip() if paragraph else None
        if abstract:
            abstract = f"{abstract}"
        
        # Ambil tahun
        year_tag = soup.find('span', class_='person_name')
        if year_tag and year_tag.next_sibling:
            sibling_text = year_tag.next_sibling.strip()
            match = re.search(r'\((\d{4})\)', sibling_text)
            year = match.group(1) if match else None
        else:
            year = None

        # Ambil universitas
        paragraphs = soup.find_all('p')
        university = None
        for para in paragraphs:
            if "IAIN" in para.text:
                university = para.text.split(",")[-1].strip()
                break
                    
        # Ambil judul
        title_tag = soup.find('title')
        title = title_tag.text.strip() if title_tag else None

        return {
            "id": id_link,
            "year": year,
            "university": university,
            "title": title,
            "abstract": abstract
        }
    else:
        print(f"Failed to fetch page for ID {id_link}. Status code: {response.status_code}")
        return {"id": id_link, "year": None, "university": None, "title": None, "abstract": None}

# Langkah 1: Scrape semua ID dari halaman utama
ids = scrape_ids(base_url)

# Langkah 2: Filter hanya ID berupa angka, mulai dari nomor 9 hingga maksimum 100
filtered_ids = [item for item in ids if item.isdigit()]
ids_from_0_to_100 = filtered_ids[0:100]  # Indeks dimulai dari 0

# Langkah 3: Ambil detail dari setiap link ID
result = []
for id_link in ids_from_0_to_100:
    details = scrape_detail(id_link)
    result.append(details)

# Langkah 4: Masukkan data ke dalam DataFrame
df_mtkuinssn = pd.DataFrame(result)

# Langkah 5: Tampilkan DataFrame
print(df_mtkuinssn.head())

     id  year                                        university  \
0  5624  None                      © IAIN Syekh Nurjati Cirebon   
1  8500  2022  S1 Tadris Matematika IAIN Syekh Nurjati Cirebon.   
2  8905  2021  S1 Tadris Matematika IAIN Syekh Nurjati Cirebon.   
3    32  None                      © IAIN Syekh Nurjati Cirebon   
4  6293  None                      © IAIN Syekh Nurjati Cirebon   

                                               title  \
0  Penggunaan Bahan Ajar Berbasis Multimedia Inte...   
1  PENGARUH PENGGUNAAN APLIKASI WORDWALL TERHADAP...   
2  ANALISIS KETERAMPILAN GURU MATEMATIKA SE-KABUP...   
3  Migration routes of the Common Shoveler  - IAI...   
4  Analysis of mathematics test items quality for...   

                                            abstract  
0                                               None  
1  Peningkatan kualitas proses dan hasil pembelaj...  
2  Permasalahan yang dihadapi pada masa pandemic ...  
3  This is where the abstract of this re

In [8]:
# Nama file CSV
filename = './csv/mtk_uinssn.csv'

# Cek apakah file sudah ada
if os.path.exists(filename):
    # Jika file sudah ada, baca file CSV ke dalam DataFrame
    existing_data = pd.read_csv(filename)
    
    # Tambahkan data baru ke DataFrame yang sudah ada
    updated_data = pd.concat([existing_data, df_mtkuinssn], ignore_index=True)
else:
    # Jika file belum ada, data baru menjadi DataFrame awal
    updated_data = df_mtkuinssn

# Simpan DataFrame gabungan ke file CSV
updated_data.to_csv(filename, index=False)

print(f"DataFrame telah diperbarui dan disimpan ke file '{filename}'")

DataFrame telah diperbarui dan disimpan ke file './csv/mtk_uinssn.csv'


## Scrapping UIN Sunan Ampel Surabaya

In [21]:
# URL dari halaman yang ingin di-scrape
base_url = "https://digilib.uinsa.ac.id/view/divisions/fst=5Fmat/2024.html"  # Ganti dengan URL target

# Fungsi untuk melakukan scraping ID dari link
def scrape_ids(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Cari semua elemen <a> dengan href mengarah ke https://
        links = soup.find_all('a', href=True)
        
        # Filter hanya link yang sesuai pola dan ekstrak ID
        ids = []
        for link in links:
            href = link['href']
            if href.startswith("https://digilib.uinsa.ac.id/"):
                # Ambil ID dari URL (angka setelah domain)
                id_value = href.split('/')[-2]  # Mengambil ID sebelum tanda "/"
                ids.append(id_value)
        
        return ids
    else:
        print(f"Failed to fetch page. Status code: {response.status_code}")
        return []

# Fungsi untuk mengambil data detail dari halaman berdasarkan ID
def scrape_detail(id_value):
    # Bentuk URL menggunakan pola yang benar
    url = f"https://digilib.uinsa.ac.id/{id_value}/"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Tambahkan proses scraping di sini sesuai kebutuhan
        
        # Ambil paragraf abstrak
        paragraph = soup.find('p', class_="ep_field_para", align="justify")  # Perbaiki selector
        abstract = paragraph.text.strip() if paragraph else None
        if abstract:
            abstract = f"{abstract}"
        
        # Ambil tahun
        year_tag = soup.find('span', class_='person_name')
        if year_tag and year_tag.next_sibling:
            sibling_text = year_tag.next_sibling.strip()
            match = re.search(r'\((\d{4})\)', sibling_text)
            year = match.group(1) if match else None
        else:
            year = None

        # Ambil universitas
        paragraphs = soup.find_all('p')
        university = None
        for para in paragraphs:
            if "UIN" in para.text:
                university = para.text.split(",")[-1].strip()
                break
                    
        # Ambil judul
        title_tag = soup.find('title')
        title = title_tag.text.strip() if title_tag else None

        return {
            "id": id_link,
            "year": year,
            "university": university,
            "title": title,
            "abstract": abstract
        }
    else:
        print(f"Failed to fetch page for ID {id_link}. Status code: {response.status_code}")
        return {"id": id_link, "year": None, "university": None, "title": None, "abstract": None}

# Langkah 1: Scrape semua ID dari halaman utama
ids = scrape_ids(base_url)

# Langkah 2: Filter hanya ID berupa angka, mulai dari nomor 9 hingga maksimum 100
filtered_ids = [item for item in ids if item.isdigit()]
ids_from_0_to_100 = filtered_ids[0:100]  # Indeks dimulai dari 0

# Langkah 3: Ambil detail dari setiap link ID
result = []
for id_link in ids_from_0_to_100:
    details = scrape_detail(id_link)
    result.append(details)

# Langkah 4: Masukkan data ke dalam DataFrame
df_mtkuinsa = pd.DataFrame(result)

# Langkah 5: Tampilkan DataFrame
print(df_mtkuinsa.head())

      id  year                 university  \
0  67895  2024  UIN Sunan Ampel Surabaya.   
1  72204  2024  UIN Sunan Ampel Surabaya.   
2  67995  2024  UIN Sunan Ampel Surabaya.   
3  71334  2024  UIN Sunan Ampel Surabaya.   
4  72196  2024  UIN Sunan Ampel Surabaya.   

                                               title  \
0  Deteksi down syndrome pada anak berdasarkan ci...   
1  Optimalisasi metode K-Means clustering dengan ...   
2  Pengukuran kinerja organisasi pada BPS Kabupat...   
3  Penerapan Metode Autoregressive Distributed La...   
4  Analisis pengendalian kualitas produksi gula m...   

                                            abstract  
0  Down syndrome merupakan penyakit genetik yang ...  
1  Klub sepak bola pasti memiliki keinginan yang ...  
2  Pengukuran kinerja adalah suatu proses untuk m...  
3  Produk Domestik Regional Bruto (PDRB) mencermi...  
4  Pengendalian kualitas merupakan aspek krusial ...  


In [22]:
# Nama file CSV
filename = './csv/mtk_uinsa2.csv'

# Cek apakah file sudah ada
if os.path.exists(filename):
    # Jika file sudah ada, baca file CSV ke dalam DataFrame
    existing_data = pd.read_csv(filename)
    
    # Tambahkan data baru ke DataFrame yang sudah ada
    updated_data = pd.concat([existing_data, df_mtkuinsa], ignore_index=True)
else:
    # Jika file belum ada, data baru menjadi DataFrame awal
    updated_data = df_mtkuinsa

# Simpan DataFrame gabungan ke file CSV
updated_data.to_csv(filename, index=False)

print(f"DataFrame telah diperbarui dan disimpan ke file '{filename}'")

DataFrame telah diperbarui dan disimpan ke file './csv/mtk_uinsa2.csv'


## Scrapping UIN Syarif Hidayatullah Jakarta

In [39]:
# URL dari halaman utama yang ingin di-scrape
base_url = "https://repository.uinjkt.ac.id/dspace/handle/123456789/59/simple-search?query=&filter_field_1=dateIssued&filter_type_1=equals&filter_value_1=%5B2019+TO+2025%5D&sort_by=dc.date.issued_dt&order=asc&rpp=100&etal=0&start=200"

# Fungsi untuk melakukan scraping ID dari link
def scrape_ids(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Cari semua elemen <a> dengan href mengarah ke repository UIN
        links = soup.find_all('a', href=True)

        # Filter hanya link yang sesuai pola dan ekstrak ID
        ids = []
        for link in links:
            href = link['href']
            if href.startswith("/dspace/handle/123456789/"):
                id_value = href.split('/')[-1]  # Ambil ID dari URL
                ids.append(id_value)

        return ids
    else:
        print(f"Failed to fetch page. Status code: {response.status_code}")
        return []

# Fungsi untuk mengambil detail dari halaman berdasarkan ID
def scrape_detail(id_value):
    url = f"https://repository.uinjkt.ac.id/dspace/handle/123456789/{id_value}/"
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        rows = soup.find_all('tr')  # Ambil semua baris tabel

        # Default values jika tidak ditemukan
        year = None
        title = None
        university = None
        abstract = None

        # Loop setiap baris tabel untuk mencari metadata yang sesuai
        for row in rows:
            label = row.find('td', class_='metadataFieldLabel')
            value = row.find('td', class_='metadataFieldValue')

            if label and value:
                text_label = label.text.strip()
                text_value = value.text.strip()

                if "Issue Date" in text_label:
                    match = re.search(r'\d{4}', text_value)
                    if match:
                        year = match.group(0)  # Ambil tahun (4 digit)
                
                if "Title" in text_label:
                    title = text_value
                
                if "UIN" in text_value:  # Deteksi universitas
                    university = text_value

                if "Abstract" in text_label:
                    abstract = text_value

        return {
            "id": id_value,
            "year": year,
            "university": university,
            "title": title,
            "abstract": abstract
        }
    else:
        print(f"Failed to fetch page for ID {id_value}. Status code: {response.status_code}")
        return {"id": id_value, "year": None, "university": None, "title": None, "abstract": None}

# Langkah 1: Scrape semua ID dari halaman utama
ids = scrape_ids(base_url)

# Langkah 2: Filter hanya ID berupa angka (maksimum 100)
filtered_ids = [item for item in ids if item.isdigit()]
ids_from_0_to_100 = filtered_ids[:100]  # Ambil maksimal 100 ID

# Langkah 3: Ambil detail dari setiap ID
result = []
for id_link in ids_from_0_to_100:
    details = scrape_detail(id_link)
    result.append(details)

# Langkah 4: Masukkan data ke dalam DataFrame
df_mtkuinshj = pd.DataFrame(result)

# Langkah 5: Tampilkan DataFrame
print(df_mtkuinshj.head())

      id  year                                         university  \
0  76184  2024  Fakultas Sains dan Teknologi UIN Syarif HIdaya...   
1  76238  2024                                               None   
2  76263  2024  Fakultas Sains dan Teknologi UIN Syarif HIdaya...   
3  76264  2024  Fakultas Sains dan Teknologi UIN Syarif Hidaya...   
4  76351  2024  Fakultas Sains dan Teknologi UIN Syarif HIdaya...   

                                               title  \
0  Perhitungan premi asuransi jiwa berjangka tanp...   
1  Pengaruh Early Warning System, Return On Asset...   
2  Solusi model fluida persamaan pelat termoelast...   
3  Operator solusi sistem persamaan Resolvent flu...   
4  Penerapan model singular spectrum analysis dal...   

                                            abstract  
0  Premi adalah sejumlah uang yang harus dibayark...  
1  Untuk menumbuhkan kepercayaan nasabah pada ind...  
2  Pelat termoelastik adalah struktur datar tipis...  
3  Pada penelitian ini, diba

In [40]:
# Nama file CSV
filename = './csv/mtk_uinshj2.csv'

# Cek apakah file sudah ada
if os.path.exists(filename):
    # Jika file sudah ada, baca file CSV ke dalam DataFrame
    existing_data = pd.read_csv(filename)
    
    # Tambahkan data baru ke DataFrame yang sudah ada
    updated_data = pd.concat([existing_data, df_mtkuinshj], ignore_index=True)
else:
    # Jika file belum ada, data baru menjadi DataFrame awal
    updated_data = df_mtkuinshj

# Simpan DataFrame gabungan ke file CSV
updated_data.to_csv(filename, index=False)

print(f"DataFrame telah diperbarui dan disimpan ke file '{filename}'")

DataFrame telah diperbarui dan disimpan ke file './csv/mtk_uinshj2.csv'


## Scrapping UIN Sumatera Utara

In [18]:
# URL dari halaman yang ingin di-scrape
base_url = "http://repository.uinsu.ac.id/view/divisions/SkripsiMAT/2024.html"  # Ganti dengan URL target

# try:
#     response = requests.get(base_url, timeout=30)  # Timeout diperpanjang menjadi 30 detik
#     print(response.text)
# except requests.exceptions.Timeout:
#     print("Request timed out. Coba lagi nanti.")
# except requests.exceptions.RequestException as e:
#     print(f"Error: {e}")

# Fungsi untuk melakukan scraping ID dari link
def scrape_ids(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Cari semua elemen <a> dengan href mengarah ke https://
        links = soup.find_all('a', href=True)
        
        # Filter hanya link yang sesuai pola dan ekstrak ID
        ids = []
        for link in links:
            href = link['href']
            if href.startswith("http://repository.uinsu.ac.id/"):
                # Ambil ID dari URL (angka setelah domain)
                id_value = href.split('/')[-2]  # Mengambil ID sebelum tanda "/"
                ids.append(id_value)
        
        return ids
    else:
        print(f"Failed to fetch page. Status code: {response.status_code}")
        return []

# Fungsi untuk mengambil data detail dari halaman berdasarkan ID
def scrape_detail(id_value):
    # Bentuk URL menggunakan pola yang benar
    url = f"http://repository.uinsu.ac.id/{id_value}/"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Tambahkan proses scraping di sini sesuai kebutuhan
        
        # Ambil paragraf abstrak
        paragraph = soup.find('p', style="text-align: left; margin: 1em auto 0em auto")  # Perbaiki selector
        abstract = paragraph.text.strip() if paragraph else None
        if abstract:
            abstract = f"{abstract}"
        
        # Ambil tahun
        year_tag = soup.find('span', class_='person_name')
        if year_tag and year_tag.next_sibling:
            sibling_text = year_tag.next_sibling.strip()
            match = re.search(r'\((\d{4})\)', sibling_text)
            year = match.group(1) if match else None
        else:
            year = None

        # Ambil universitas
        paragraphs = soup.find_all('p')
        university = None
        for para in paragraphs:
            if "UIN" in para.text:
                university = para.text.split(",")[-1].strip()
                break
                    
        # Ambil judul
        title_tag = soup.find('em')
        title = title_tag.text.strip() if title_tag else None

        return {
            "id": id_link,
            "year": year,
            "university": university,
            "title": title,
            "abstract": abstract
        }
    else:
        print(f"Failed to fetch page for ID {id_link}. Status code: {response.status_code}")
        return {"id": id_link, "year": None, "university": None, "title": None, "abstract": None}

# Langkah 1: Scrape semua ID dari halaman utama
ids = scrape_ids(base_url)

# Langkah 2: Filter hanya ID berupa angka, mulai dari nomor 9 hingga maksimum 100
filtered_ids = [item for item in ids if item.isdigit()]
ids_from_0_to_100 = filtered_ids[0:100]  # Indeks dimulai dari 0

# Langkah 3: Ambil detail dari setiap link ID
result = []
for id_link in ids_from_0_to_100:
    details = scrape_detail(id_link)
    result.append(details)

# Langkah 4: Masukkan data ke dalam DataFrame
df_mtkuinsumut = pd.DataFrame(result)

# Langkah 5: Tampilkan DataFrame
print(df_mtkuinsumut.head())

      id  year                                         university  \
0  22810  2024  © 2022 PUSTIPADA UIN Sumatera Utara - Powered ...   
1  23131  2024                                UIN Surmatra Utara.   
2  23896  2024                          UIN Sumatera Utara Medan.   
3  24229  2024                          UIN Sumatera Utara Medan.   
4  24457  2024                                UIN Sumatera Utara.   

                                               title  \
0  PENGENDALIAN KUALITAS AIR SUNGAI DELI KOTA\r\n...   
1  Penerapan Metode Multi Attribute Utility Theor...   
2  Penerapan Partial Least Square-Modified Fuzzy\...   
3  Penerapan Algoritma Tabu Search Pada Capacitat...   
4  Penerapan Metode Chi-Square Automatic\r\nInter...   

                                            abstract  
0  Clean water is a type of water-based resource ...  
1  Beasiswa UPZ merupakan beasiswa yang bersumber...  
2  JNE Express merupakan salah satu perusahaan ya...  
3  Kegiatan pengangkutan sam

In [19]:
# Nama file CSV
filename = './csv/mtk_uinsumut2.csv'

# Pastikan folder tempat menyimpan file ada
os.makedirs(os.path.dirname(filename), exist_ok=True)

try:
    # Cek apakah file sudah ada
    if os.path.exists(filename):
        # Jika file sudah ada, baca file CSV ke dalam DataFrame
        existing_data = pd.read_csv(filename, encoding="utf-8-sig")
        
        # Tambahkan data baru ke DataFrame yang sudah ada
        updated_data = pd.concat([existing_data, df_mtkuinsumut], ignore_index=True)
    else:
        # Jika file belum ada, data baru menjadi DataFrame awal
        updated_data = df_mtkuinsumut

    # Simpan DataFrame gabungan ke file CSV
    updated_data.to_csv(filename, index=False, encoding="utf-8-sig")

    print(f"DataFrame telah diperbarui dan disimpan ke file '{filename}'")

except Exception as e:
    print(f"Terjadi kesalahan: {e}")

DataFrame telah diperbarui dan disimpan ke file './csv/mtk_uinsumut2.csv'


## UIN Maulana Malik Ibrahim Malang

In [38]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from lxml import html

# URL dari halaman yang ingin di-scrape
base_url = "http://etheses.uin-malang.ac.id/view/divisions/JMat/2024.html"  # Ganti dengan URL target

# Fungsi untuk melakukan scraping ID dari link
def scrape_ids(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Cari semua elemen <a> dengan href mengarah ke halaman detail
        links = soup.find_all('a', href=True)
        
        # Filter hanya link yang sesuai pola dan ekstrak ID
        ids = []
        for link in links:
            href = link['href']
            if href.startswith("http://etheses.uin-malang.ac.id/"):
                id_value = href.split('/')[-2]  # Mengambil ID dari URL
                ids.append(id_value)
        
        return ids
    else:
        print(f"Failed to fetch page. Status code: {response.status_code}")
        return []

# Fungsi untuk mengambil data detail dari halaman berdasarkan ID
def scrape_detail(id_value):
    url = f"http://etheses.uin-malang.ac.id/{id_value}/"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Ambil paragraf abstrak menggunakan XPath
        tree = html.fromstring(response.content)
        xpath_expression = '//p[@class="ep_field_para" and @align="justify"]/text()'
        abstract_list = tree.xpath(xpath_expression)
        abstract = " ".join(abstract_list).strip() if abstract_list else None
        
        # Ambil tahun
        year_tag = soup.find('span', class_='person_name')
        if year_tag and year_tag.next_sibling:
            sibling_text = year_tag.next_sibling.strip()
            match = re.search(r'\((\d{4})\)', sibling_text)
            year = match.group(1) if match else None
        else:
            year = None

        # Ambil universitas
        paragraphs = soup.find_all('p')
        university = None
        for para in paragraphs:
            if "Universitas" in para.text:
                university = para.text.split(",")[-1].strip()
                break
                    
        # Ambil judul
        title_tag = soup.find('em')
        title = title_tag.text.strip() if title_tag else None

        return {
            "id": id_value,
            "year": year,
            "university": university,
            "title": title,
            "abstract": abstract
        }
    else:
        print(f"Failed to fetch page for ID {id_value}. Status code: {response.status_code}")
        return {"id": id_value, "year": None, "university": None, "title": None, "abstract": None}

# Langkah 1: Scrape semua ID dari halaman utama
ids = scrape_ids(base_url)

# Langkah 2: Filter hanya ID berupa angka, mulai dari nomor 0 hingga maksimum 100
filtered_ids = [item for item in ids if item.isdigit()]
ids_from_0_to_100 = filtered_ids[:100]  # Ambil maksimum 100 ID

# Langkah 3: Ambil detail dari setiap ID
result = []
for id_link in ids_from_0_to_100:
    details = scrape_detail(id_link)
    result.append(details)

# Langkah 4: Masukkan data ke dalam DataFrame
df_mtkuinmalik = pd.DataFrame(result)

# Langkah 5: Tampilkan DataFrame
print(df_mtkuinmalik.head())


      id  year                                       university  \
0  71316  2024  Universitas Islam Negeri Maulana Malik Ibrahim.   
1  65301  2024  Universitas Islam Negeri Maulana Malik Ibrahim.   
2  71223  2024  Universitas Islam Negeri Maulana Malik Ibrahim.   
3  71351  2024  Universitas Islam Negeri Maulana Malik Ibrahim.   
4  71349  2024  Universitas Islam Negeri Maulana Malik Ibrahim.   

                                               title  \
0  Impelementasi metode mean shift untuk clusteri...   
1  Identifikasi autokorelasi spasial sebaran peti...   
2  Implementasi algoritma support vector machine ...   
3  Randic, Sum-Connectivity Indeks dan koindeks p...   
4  Randic, sum-connectivity indeks dan koindeks p...   

                                            abstract  
0  INDONESIA : Identifikasi daerah rawan kejadian...  
1  ABSTRAK Data spasial merupakan data dengan asp...  
2  INDONESIA: Diabetes Mellitus adalah penyakit k...  
3  INDONESIA: Penelitian ini mengkaji te

In [39]:
# Nama file CSV
filename = './csv/mtk_uinmalik2.csv'

# Pastikan folder tempat menyimpan file ada
os.makedirs(os.path.dirname(filename), exist_ok=True)

try:
    # Cek apakah file sudah ada
    if os.path.exists(filename):
        # Jika file sudah ada, baca file CSV ke dalam DataFrame
        existing_data = pd.read_csv(filename, encoding="utf-8-sig")
        
        # Tambahkan data baru ke DataFrame yang sudah ada
        updated_data = pd.concat([existing_data, df_mtkuinmalik], ignore_index=True)
    else:
        # Jika file belum ada, data baru menjadi DataFrame awal
        updated_data = df_mtkuinmalik

    # Simpan DataFrame gabungan ke file CSV
    updated_data.to_csv(filename, index=False, encoding="utf-8-sig")

    print(f"DataFrame telah diperbarui dan disimpan ke file '{filename}'")

except Exception as e:
    print(f"Terjadi kesalahan: {e}")

DataFrame telah diperbarui dan disimpan ke file './csv/mtk_uinmalik2.csv'


## Scrapping UIN Walisongo

In [12]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from lxml import html

# URL dari halaman yang ingin di-scrape
base_url = "https://eprints.walisongo.ac.id/view/divisions/jur=5Fmat/2024.html"  # Ganti dengan URL target

# Fungsi untuk melakukan scraping ID dari link
def scrape_ids(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Cari semua elemen <a> dengan href mengarah ke halaman detail
        links = soup.find_all('a', href=True)
        
        # Filter hanya link yang sesuai pola dan ekstrak ID
        ids = []
        for link in links:
            href = link['href']
            if href.startswith("https://eprints.walisongo.ac.id/"):
                id_value = href.split('/')[-2]  # Mengambil ID dari URL
                ids.append(id_value)
        
        return ids
    else:
        print(f"Failed to fetch page. Status code: {response.status_code}")
        return []

# Fungsi untuk mengambil data detail dari halaman berdasarkan ID
def scrape_detail(id_value):
    url = f"https://eprints.walisongo.ac.id/id/eprint/{id_value}/"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Ambil paragraf abstrak menggunakan XPath
        tree = html.fromstring(response.content)
        xpath_expression = '//*[@id="eprints_content"]/div/div[4]/div[3]'
        abstract_list = tree.xpath(xpath_expression)
        abstract = " ".join([elem.text_content().strip() for elem in abstract_list]) if abstract_list else None

        # Ambil paragraf abstrak
        # paragraph = soup.find('p', style="text-align: left; margin: 1em auto 0em auto")  # Perbaiki selector
        # abstract = paragraph.text.strip() if paragraph else None
        # if abstract:
        #     abstract = f"{abstract}"
        
        # Ambil tahun
        year_tag = soup.find('span', class_='person_name')
        if year_tag and year_tag.next_sibling:
            sibling_text = year_tag.next_sibling.strip()
            match = re.search(r'\((\d{4})\)', sibling_text)
            year = match.group(1) if match else None
        else:
            year = None

        # Ambil universitas
        paragraphs = soup.find_all('p')
        university = None
        for para in paragraphs:
            if "Universitas" in para.text:
                university = para.text.split(",")[-1].strip()
                break
                    
        # Ambil judul
        title_tag = soup.find('em')
        title = title_tag.text.strip() if title_tag else None

        return {
            "id": id_value,
            "year": year,
            "university": university,
            "title": title,
            "abstract": abstract
        }
    else:
        print(f"Failed to fetch page for ID {id_value}. Status code: {response.status_code}")
        return {"id": id_value, "year": None, "university": None, "title": None, "abstract": None}

# Langkah 1: Scrape semua ID dari halaman utama
ids = scrape_ids(base_url)

# Langkah 2: Filter hanya ID berupa angka, mulai dari nomor 0 hingga maksimum 100
filtered_ids = [item for item in ids if item.isdigit()]
ids_from_0_to_100 = filtered_ids[:100]  # Ambil maksimum 100 ID

# Langkah 3: Ambil detail dari setiap ID
result = []
for id_link in ids_from_0_to_100:
    details = scrape_detail(id_link)
    result.append(details)

# Langkah 4: Masukkan data ke dalam DataFrame
df_mtkuinwalisongo = pd.DataFrame(result)

# Langkah 5: Tampilkan DataFrame
print(df_mtkuinwalisongo.head())


Failed to fetch page. Status code: 404
Empty DataFrame
Columns: []
Index: []


In [11]:
# Nama file CSV
filename = './csv/mtk_walisongo2.csv'

# Pastikan folder tempat menyimpan file ada
os.makedirs(os.path.dirname(filename), exist_ok=True)

try:
    # Cek apakah file sudah ada
    if os.path.exists(filename):
        # Jika file sudah ada, baca file CSV ke dalam DataFrame
        existing_data = pd.read_csv(filename, encoding="utf-8-sig")
        
        # Tambahkan data baru ke DataFrame yang sudah ada
        updated_data = pd.concat([existing_data, df_mtkuinwalisongo], ignore_index=True)
    else:
        # Jika file belum ada, data baru menjadi DataFrame awal
        updated_data = df_mtkuinwalisongo

    # Simpan DataFrame gabungan ke file CSV
    updated_data.to_csv(filename, index=False, encoding="utf-8-sig")

    print(f"DataFrame telah diperbarui dan disimpan ke file '{filename}'")

except Exception as e:
    print(f"Terjadi kesalahan: {e}")

DataFrame telah diperbarui dan disimpan ke file './csv/mtk_walisongo2.csv'
